# 요구사항
1. 우리나라 대기업 국내외 관련기사->투자, 협약 맺음, 건설 내용잇음더 좋음
2. 중국기업 관련기사 국내외
3. 중국 기업이 우리나라에 투자하거나 우리나라 기업과 협력 관련기사->거의 안뜸
4. 중국 기업이 우리나라에 들어오는 기사->요즘은 알리,테무가 많이 뜨더라
이정도..?

# pip install

In [75]:
!pip3 install requests beautifulsoup4
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


# Main

## BeautifulSoup `https://news.naver.com`

In [76]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option('display.max_colwidth', 100)

root_url = "https://news.naver.com/"
href_pattern = re.compile(r'^https?://[n.]?news\.naver\.com/')

def get_news_naver_html(url: str):
    response = requests.get(url)
    if not response.status_code == 200:
        print("Failed to retrieve the webpage")
        return

    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

## 속보 크롤링

In [77]:
# 속보
def newsflash_crawling(root_html: BeautifulSoup):
    newsflash_list = root_html.find('div', class_="cjs_nf_list _item_list").find_all('a')

    if not newsflash_list:
        print('속보를 찾을 수 없습니다.')
    else:
        print('속보 테이블')
        newsflash_data = []
        for newsflash in newsflash_list:
            href = newsflash.get('href')
            title = newsflash.find('h4', class_="cn_title").text.strip()
            company_name = newsflash.find('div', class_="cn_name").text.strip()
            newsflash_data.append({'topic':'속보', 'title': title, 'company_name': company_name, 'link': href})
        
        df = pd.DataFrame(newsflash_data)
        return df

df_newsflash = newsflash_crawling(get_news_naver_html(root_url))
df_newsflash

속보 테이블


,topic,title,company_name,link
0,속보,"부산 민주당, '환호'하다가도 부산 출구조사선 '우려'",국제신문,https://n.news.naver.com/article/658/0000070986
1,속보,"‘막말 논란·편법대출’ 수원정 김준혁 우세, 안산갑 양문석 경합",문화일보,https://n.news.naver.com/article/021/0002631383
2,속보,개표 10.0% 현재 민주 80곳·국민의힘 129곳 지역구 1위,헤럴드경제,https://n.news.naver.com/article/016/0002293105
3,속보,출구조사 희비…민주 ‘축제 분위기’ 국힘 ‘대충격에 침통’,부산일보,https://n.news.naver.com/article/082/0001264653


## 경제 섹션 크롤링

In [78]:
def economy_crawling(root_html: BeautifulSoup):
    economy_list = root_html.find('div', id="newsct").find_all('li')
    
    if not economy_list:
        print('경제 섹션을 찾을 수 없습니다.')
    else:
        print('경제 섹션 테이블')
        economy_data = []

        for economy in economy_list:
            content_html = economy.find('div', class_="sa_text")
            if not content_html:
                continue
            href = content_html.find('a', class_="sa_text_title").get('href')
            title = content_html.find('strong', class_="sa_text_strong").text.strip()
            description = content_html.find('div', class_="sa_text_lede").text.strip()
            company_name = content_html.find('div', class_="sa_text_press").text.strip()
            
            economy_data.append({'topic':'경제', 'title': title, 'company_name': company_name, "description": description,'link': href})
        
        df = pd.DataFrame(economy_data)
        return df

df_economy = economy_crawling(get_news_naver_html("https://news.naver.com/section/101"))
df_economy

경제 섹션 테이블


,topic,title,company_name,description,link
0,경제,안덕근 산업장관 첫 방미...IRA·반도체법 논의,한국일보,"안덕근 산업통상자원부 장관이 취임 후 처음으로 미국을 방문했다. 안 장관은 미국 당국자들과 인플레이션 감축법(IRA), 반도체법 등 통상 현안을 논의할 계획이다. 산업통상자...",https://n.news.naver.com/mnews/article/469/0000795223
1,경제,10년간 50만명 다녀간 삼성 이노베이션뮤지엄,매일경제,개관 10돌 180개국서 방문 한국전자산업 역사 한눈에 전자산업의 역사를 한눈에 살펴볼 수 있는 삼성전자의 '삼성 이노베이션 뮤지엄'이 이달로 개관 10주년을 맞았다. 10...,https://n.news.naver.com/mnews/article/009/0005286201
2,경제,"삼성전자서비스, 로봇청소기 ‘비스포크 AI 스팀’ 고객 서비스 강화",디지털타임스,삼성전자서비스는 지난 9일부터 서비스 엔지니어를 대상으로 '비스포크 AI 스팀 신제품 교육'을 진행하고 있다고 10일 밝혔다. 삼성전자서비스는 비스포크 AI 스팀에 국내 최...,https://n.news.naver.com/mnews/article/029/0002866526
3,경제,"""민생 안정과 경제 활력 위한 초당적 협력을"" 한경협, 총선 논평",파이낸셜뉴스,"한국경제인협회는 10일 제22대 국회가 이번 총선에서 나타난 국민의 뜻을 겸허히 받아들이고, 상생과 화합의 정신으로 민생경제 안정과 경제 활력을 되살리는 데 혼신의 힘을 다...",https://n.news.naver.com/mnews/article/014/0005169479
4,경제,"조만호號 무신사, 나이키 입점…플랫폼 경쟁력 제고",전자신문,무신사가 대형 스포츠 브랜드 나이키를 유치하며 플랫폼 경쟁력을 강화한다. 10일 업계에 따르면 무신사는 최근 나이키 유치에 성공했다. 이에따라 오는 15일부터 나이키가 무신...,https://n.news.naver.com/mnews/article/030/0003196780
5,경제,"'한강뷰' 한남5구역·서울역 39층 랜드마크, 서울시 심의 통과",한국일보,"서울 용산구 일대 재개발사업(한남뉴타운) 중 가장 입지가 좋은 곳으로 꼽히는 한남5구역에 남산과 한강을 볼 수 있는 2,592세대 규모 아파트 단지 건축 계획이 확정됐다. ...",https://n.news.naver.com/mnews/article/469/0000795213
6,경제,"""올해도 투자환경 어려워...현금흐름 중심 경영에 집중해야""",머니투데이,"최대 16.5억원의 투자기회가 주어진 제13회 청년기업가대회의 심사위원으로 벤처심사역뿐만 아니라 예비유니콘 스타트업 대표, 대기업계열 복지재단 이사장 등 총 27인의 다양한...",https://n.news.naver.com/mnews/article/008/0005023450
7,경제,작년 대기업 직원은 줄고 임원은 늘어…임원 1인당 직원 107.6명,연합뉴스,리더스인덱스 분석…은행·유통 등 전년대비 비율 감소 커 지난해 국내 주요 대기업 직원 수는 전체적으로 감소한 반면 임원은 늘어 임원 1인당 직원 비율이 전년보다 줄어든 것으...,https://n.news.naver.com/mnews/article/001/0014621637
8,경제,"美물가 반등 경계…환율 1354.9원, 3거래일째 연고점 경신[외환마감]",이데일리,원·달러 환율이 1350원 중반대에 안착하며 3거래일째 연고점을 경신했다. 미국 소비자물가(CPI) 상승 경계감에 환율이 제한적 상승에 그쳤다. 9일 오후 서울 중구 하나은...,https://n.news.naver.com/mnews/article/018/0005710856
9,경제,롤러코스터 타는 비트코인…하룻밤새 7.2만→6.8만달러 급락,디지털타임스,글로벌 금융시장의 불확실성이 커지는 가운데 가상화폐 대장주 비트코인 가격도 롤러코스터를 타고 있다. 미국 3월 소비자물가지수(CPI) 발표를 앞두고 관망세가 짙어진 가운데 ...,https://n.news.naver.com/mnews/article/029/0002866498


## join df

In [79]:
total_news_df = pd.concat([df_newsflash, df_economy])
total_news_df

,topic,title,company_name,link,description
0,속보,"부산 민주당, '환호'하다가도 부산 출구조사선 '우려'",국제신문,https://n.news.naver.com/article/658/0000070986,NaN
1,속보,"‘막말 논란·편법대출’ 수원정 김준혁 우세, 안산갑 양문석 경합",문화일보,https://n.news.naver.com/article/021/0002631383,NaN
2,속보,개표 10.0% 현재 민주 80곳·국민의힘 129곳 지역구 1위,헤럴드경제,https://n.news.naver.com/article/016/0002293105,NaN
3,속보,출구조사 희비…민주 ‘축제 분위기’ 국힘 ‘대충격에 침통’,부산일보,https://n.news.naver.com/article/082/0001264653,NaN
0,경제,안덕근 산업장관 첫 방미...IRA·반도체법 논의,한국일보,https://n.news.naver.com/mnews/article/469/0000795223,"안덕근 산업통상자원부 장관이 취임 후 처음으로 미국을 방문했다. 안 장관은 미국 당국자들과 인플레이션 감축법(IRA), 반도체법 등 통상 현안을 논의할 계획이다. 산업통상자..."
1,경제,10년간 50만명 다녀간 삼성 이노베이션뮤지엄,매일경제,https://n.news.naver.com/mnews/article/009/0005286201,개관 10돌 180개국서 방문 한국전자산업 역사 한눈에 전자산업의 역사를 한눈에 살펴볼 수 있는 삼성전자의 '삼성 이노베이션 뮤지엄'이 이달로 개관 10주년을 맞았다. 10...
2,경제,"삼성전자서비스, 로봇청소기 ‘비스포크 AI 스팀’ 고객 서비스 강화",디지털타임스,https://n.news.naver.com/mnews/article/029/0002866526,삼성전자서비스는 지난 9일부터 서비스 엔지니어를 대상으로 '비스포크 AI 스팀 신제품 교육'을 진행하고 있다고 10일 밝혔다. 삼성전자서비스는 비스포크 AI 스팀에 국내 최...
3,경제,"""민생 안정과 경제 활력 위한 초당적 협력을"" 한경협, 총선 논평",파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/0005169479,"한국경제인협회는 10일 제22대 국회가 이번 총선에서 나타난 국민의 뜻을 겸허히 받아들이고, 상생과 화합의 정신으로 민생경제 안정과 경제 활력을 되살리는 데 혼신의 힘을 다..."
4,경제,"조만호號 무신사, 나이키 입점…플랫폼 경쟁력 제고",전자신문,https://n.news.naver.com/mnews/article/030/0003196780,무신사가 대형 스포츠 브랜드 나이키를 유치하며 플랫폼 경쟁력을 강화한다. 10일 업계에 따르면 무신사는 최근 나이키 유치에 성공했다. 이에따라 오는 15일부터 나이키가 무신...
5,경제,"'한강뷰' 한남5구역·서울역 39층 랜드마크, 서울시 심의 통과",한국일보,https://n.news.naver.com/mnews/article/469/0000795213,"서울 용산구 일대 재개발사업(한남뉴타운) 중 가장 입지가 좋은 곳으로 꼽히는 한남5구역에 남산과 한강을 볼 수 있는 2,592세대 규모 아파트 단지 건축 계획이 확정됐다. ..."
